In [17]:

import pandas as pd
data.row = pd.read_csv("../data/data/aapl_raw_data.csv")
#print(data.row.iloc[10748])
print(data.row.loc[10748, 'close'])
data.row.loc[10748, 'close']



192.58


C:\Users\pitr7\AppData\Local\Temp\ipykernel_10032\1004640409.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data.row = pd.read_csv("../data/data/aapl_raw_data.csv")


192.58

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

print(data.tail(1))

             date    open    high     low   close      volume  adjusted_close  \
10746  2023-07-31  196.06  196.49  195.26  196.45  38824100.0        196.1851   

       change_percent  avg_vol_20d  
10746            0.32   49803320.0  


Transform data to data-embeding
Split the data into training and testing sets
fit_transform: x_train, y_train 
    transform: x_test, y_test

In [11]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D



# Convert data to Tensors
x_feature_tensors = torch.tensor(x_data, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data, dtype=torch.float32)


# Split the data into training and testing sets
train_size = int(len(x_feature_tensors) * 0.67)
x_train, x_test = x_feature_tensors[:train_size], x_feature_tensors[train_size:]
y_train, y_test = y_feature_tensors[:train_size], y_feature_tensors[train_size:]



# Initialize the scaler
scaler = MinMaxScaler()

# fitting the MinMaxScaler to training data using fit_transform and then
#  test data only transform
x_train_scaled = scaler.fit_transform(x_train.detach().numpy())
x_test_scaled = scaler.transform(x_test.detach().numpy())  # Use transform, not fit_transform

y_train_scaled = scaler.fit_transform(y_train.detach().numpy())
y_test_scaled = scaler.transform(y_test.detach().numpy())  # Use transform, not fit_transform





LSTM Model 
Initialize the model, loss, and optimizer
Training using Walk-Forward Validation
Evaluate the model on the test set

In [18]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

# Adjustable Parameters
learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train_scaled) - window_size
batch_size = 1
input_size = x_train_scaled.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

# LSTM Model Definition
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

# Initialize the model, loss, and optimizer
model = LSTMModel(input_size, hidden_dim, n_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training using Walk-Forward Validation
for i in range(window_size, epochs):
    optimizer.zero_grad()

    # Extract training window with batch size
    start_idx = i - window_size
    end_idx = i
    x_window = torch.tensor(x_train_scaled[start_idx:end_idx], dtype=torch.float32)
    y_window = torch.tensor(y_train_scaled[start_idx:end_idx], dtype=torch.float32)

    # Reshape x_window to match LSTM input shape [batch_size=1, sequence_length=window_size, input_size=input_size]
    x_window = x_window.view(1, window_size, input_size)

    # Initialize hidden state
    hidden = model.init_hidden(batch_size)

    # Forward pass
    outputs, hidden = model(x_window, hidden)
    loss = criterion(outputs, y_window)

    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    # Evaluate the model on the test set
    if i % batch_evaluation_frequency == 0:
        with torch.no_grad():
            x_test_window = torch.tensor(x_test_scaled[:test_window_size], dtype=torch.float32)
            y_test_window = torch.tensor(y_test_scaled[:test_window_size], dtype=torch.float32)

            # Reshape x_test_window to match LSTM input shape [batch_size=1, sequence_length=test_window_size, input_size=input_size]
            x_test_window = x_test_window.view(1, test_window_size, input_size)

            # Initialize hidden state for test
            hidden = model.init_hidden(batch_size)

            test_outputs, _ = model(x_test_window, hidden)
            test_loss = criterion(test_outputs, y_test_window)
            print(f"Iteration {i}, Train Loss: {loss.item()}, Test Loss: {test_loss.item()}")






c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([50, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Iteration 50, Train Loss: 0.007849492132663727, Test Loss: 0.5407178401947021
Iteration 60, Train Loss: 0.0004204017750453204, Test Loss: 0.465374231338501
Iteration 70, Train Loss: 0.00034028541995212436, Test Loss: 0.4879387319087982
Iteration 80, Train Loss: 0.00019509323465172201, Test Loss: 0.4792060852050781
Iteration 90, Train Loss: 0.00010191080946242437, Test Loss: 0.49910080432891846
Iteration 100, Train Loss: 9.931065869750455e-05, Test Loss: 0.48317354917526245
Iteration 110, Train Loss: 9.318694355897605e-05, Test Loss: 0.4868715703487396
Iteration 120, Train Loss: 0.0001489044661866501, Test Loss: 0.4700486660003662
Iteration 130, Train Loss: 0.00013791528181172907, Test Loss: 0.46521449089050293
Iteration 140, Train Loss: 0.00012569177488330752, Test Loss: 0.46031609177589417
Iteration 150, Train Loss: 0.00025364040629938245, Test Loss: 0.46982234716415405
Iteration 160, Train Loss: 0.00035947837750427425, Test Loss: 0.4721713662147522
Iteration 170, Train Loss: 0.000277

In [20]:
# Extract the last test_window_size data points from the scaled test data
last_test_window = x_test_scaled[-test_window_size:]

# Convert the last_test_window to a PyTorch tensor and reshape it
last_test_window_tensor = torch.tensor(last_test_window, dtype=torch.float32).view(1, test_window_size, input_size)

# Initialize hidden state for prediction
hidden = model.init_hidden(1)  # Batch size of 1

# Perform forward pass to get the predicted close value for the next day
with torch.no_grad():
    predicted_close_scaled, _ = model(last_test_window_tensor, hidden)

# Inverse transform the predicted close value to get the actual predicted close value
predicted_close = scaler.inverse_transform(predicted_close_scaled.detach().numpy())

print(f"Predicted Close Value for the Next Day: {predicted_close[0][0]}")


# Real value from the data row
data.row = pd.read_csv("../data/data/aapl_raw_data.csv")
real_value = data.row.loc[10748, 'close']

# Calculate the error
predicted_error  = real_value - predicted_close[0][0]


# Print the real value and error
print("Real close price:", real_value)
print("Error predicted value:", predicted_error)



Predicted Close Value for the Next Day: 101.36563110351562
Real close price: 192.58
Error predicted value: 91.21436889648439


In [ ]:
# Real value
real_value = data.row.loc[10748, 'close']

# Calculate the squared error
squared_error = (real_value - predicted_value[0][0]) ** 2

print("Real close price:", real_value)
print("Squared Error (MSE):", squared_error)


Real close price: 192.58
Squared Error (MSE): 8558.91500643372


In [ ]:


# Set the model to evaluation mode
model.eval()

# Initialize hidden state
hidden = model.init_hidden(1)  # Batch size is 1

# Initialize the input sequence with the last window_size data points from the training set
input_sequence = x_train_scaled[-window_size:].tolist()


# Get prediction from model
prediction, hidden = model(input_tensor, hidden)

# Get prediction from model
prediction, hidden = model(input_tensor, hidden)

# Inverse scaling the predicted value
predicted_value_scaled = prediction.item()  # Get the scaled predicted value
predicted_value = scaler.inverse_transform([[predicted_value_scaled]])  # Inverse transform to get the actual predicted value

# Print the actual predicted value
print("Next time step predicted close price:", predicted_value[0][0])



Next time step predicted close price: 100.06559568135502


In [ ]:
# Number of future time steps you want to predict
num_future_steps = 10

# Set the model to evaluation mode
model.eval()

# Initialize hidden state
hidden = model.init_hidden(1)  # Batch size is 1

# Initialize the input sequence with the last window_size data points from the training set
input_sequence = x_train_scaled[-window_size:].tolist()

future_predictions = []

with torch.no_grad():
    for _ in range(num_future_steps):
        # Convert the input_sequence to a tensor and reshape
        input_tensor = torch.tensor([input_sequence], dtype=torch.float32)
        input_tensor = input_tensor.view(1, window_size, input_size)

        # Get prediction from model
        prediction, hidden = model(input_tensor, hidden)

        # Append the prediction to future_predictions
        future_predictions.append(prediction.item())

        # Update input_sequence with the new prediction (for the next iteration)
        # Note: For simplicity, we're updating only one feature, assuming it's the 'close' value.
        # If you have multiple features in the input_sequence, adjust accordingly.
        input_sequence.append(prediction.item())
        input_sequence = input_sequence[-window_size:]  # Keep only the last window_size elements

# Inverse transform the scaled predictions to get the actual predicted "close" values
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

# Print or use future_predictions as needed
print(future_predictions)


TypeError: not a sequence